## TP2 Groupe 12

- Châkirou ALABANI
- Caroline Fortier
- Mahsa Farnia

In [2]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib notebook
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

## 1.  Analyse des données


In [6]:
#import des données 
# 
file_a = "données/order_products__prior_specials.csv"
data_a = pd.read_csv(file_a)

# dimensions de notre dataframe
data_a.shape #

(1172312, 6)

In [8]:
# affichage du describe et du head
data_a.describe

<bound method NDFrame.describe of          Unnamed: 0  order_id  product_id  add_to_cart_order  reordered  \
0                 0        15       19660                  1          1   
1                 1        15       21195                  2          1   
2                 2        15        7461                  3          1   
3                 3        15        2996                  4          1   
4                 4        15       32463                  5          1   
...             ...       ...         ...                ...        ...   
1172307     1172307   3421069       21137                  8          1   
1172308     1172308   3421069       30391                  9          1   
1172309     1172309   3421069       18656                 10          1   
1172310     1172310   3421069       26940                 11          1   
1172311     1172311   3421069       19057                 12          1   

         special  
0             15  
1              0  
2       